## Average stats across all high-quality waveforms for each event

### Import libraries

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import os
import glob
import sys

### Load catalog of high quality isolated and repeating detections

In [2]:
mqdir = '../catalogs/final_catalogs/'
cat = pd.read_csv(mqdir + 'A17_moonquakes_catalog_HQ_final.csv')
cat.drop(list(cat.filter(regex='Unnamed|index')), axis=1, inplace=True)
evids = np.unique(cat.evid.tolist())

### Iterate through events and average waveform characteristics

In [3]:
avg_picktimes = []
temps_reg_avg = []
temps_rock_avg = []
int_day_nums_avg = []
pgvs_avg = []
durations_avg = []
emergences_avg = []
azimuths_lag = []
sds_lag = []
azimuths_snr = []
sds_snr = []
grades_new = []
isol_or_rpt = []
evid_refs = []

# Iterate
for evid in evids:
    rows = cat.loc[cat.evid == evid]
    
    # Average temperatures, durations, PGV, emergence
    avg_temp_reg = np.mean(np.array(rows.Regolith_Temp_K.tolist()))
    avg_temp_rock = np.mean(np.array(rows.Rock_Temp_K.tolist()))
    avg_day_num = np.mean(np.array(rows.interval_day_number.tolist()))
    avg_PGV = np.mean(np.array(rows.PGV.tolist()))
    avg_duration = np.mean(np.array(rows.Rock_Temp_K.tolist()))
    avg_emergence = np.mean(np.array(rows.emergence_s.tolist()))
    
    # Average pick time
    picktimes = [datetime.strptime(mqdate, '%Y-%m-%d %H:%M:%S.%f') for mqdate in rows.picktime_SNR.tolist()]
    mintime = min(picktimes)
    timedeltas = []
    for a in np.arange(0,len(picktimes)):
        arrtime = picktimes[a]
        timedeltas.append((arrtime - mintime).total_seconds())
    #
    mean_timedelta = np.mean(timedeltas)
    mean_time = mintime + timedelta(seconds = mean_timedelta)
    
    # Obtain azimuths (already same across geophones)
    az_lag = np.array(rows.theta_mean_lag_deg.tolist())[0]
    sd_lag = np.array(rows.theta_sd_lag_deg.tolist())[0]
    az_snr = np.array(rows.theta_mean_snr_deg.tolist())[0]
    sd_snr = np.array(rows.theta_sd_snr_deg.tolist())[0]
    
    # Grade + isolated or repeating
    grade_new = np.array(rows.grade_new.tolist())[0]
    classification = np.array(rows.isol_or_rpt.tolist())[0]
    evid_ref = np.array(rows.evid_ref.tolist())[0]
    
    # Append values to arrays
    avg_picktimes.append(mean_time)
    temps_reg_avg.append(avg_temp_reg)
    temps_rock_avg.append(avg_temp_rock)
    int_day_nums_avg.append(avg_day_num)
    pgvs_avg.append(avg_PGV)
    durations_avg.append(avg_duration)
    emergences_avg.append(avg_emergence)
    azimuths_lag.append(az_lag)
    sds_lag.append(sd_lag)
    azimuths_snr.append(az_snr)
    sds_snr.append(sd_snr)
    grades_new.append(grade_new)
    isol_or_rpt.append(classification)
    evid_refs.append(evid_ref)

### Construct a new dataframe

In [4]:
d = {'evid':evids, 'evid_ref':evid_refs, 'avg_picktime_SNR':avg_picktimes, 'grade_new':grades_new, 'isol_or_rpt':isol_or_rpt, 'avg_temp_Regolith_K':temps_reg_avg, 
    'avg_temp_Rock_K':temps_rock_avg, 'avg_interval_day':int_day_nums_avg, 'avg_PGV':pgvs_avg, 'avg_duration_s':durations_avg, 
    'avg_emergence_s':emergences_avg, 'theta_mean_lag_deg':azimuths_lag, 'theta_sd_lag_deg':sds_lag, 
    'theta_mean_snr_deg':azimuths_snr, 'theta_sd_snr_deg':sds_snr}
avg_df = pd.DataFrame(data = d)
avg_df.to_csv(mqdir + 'A17_moonquakes_catalog_HQ_avg_event_stats.csv',index=False)